In [1]:
##############################################
import sys,os
import numpy as np
import pylab as py
##############################################
import pylab as py
from  matplotlib import rc
import matplotlib.pyplot as plt
%matplotlib inline  
%config InlineBackend.figure_format = 'retina'  
##############################################
PATH='../python/'
sys.path.append(PATH)  # path of the JAMLIB python folder is located
from corelib import JAMLIB
##############################################

# Code for LHAPDF grids

## JAMLIB testing

In [2]:
def testjam(x,Q2):

  print 'alphaS     = ',jamFF.get_alphaS(Q2)
  print 'no. of posteriors = ',jamFF.npos
  print 'xFF(ipos=0) = ',jamFF.get_XF(0,'up',x,Q2)

  xgrid = jamFF.get_Xgrid()
  xmin = np.min(xgrid)
  xmax = np.max(xgrid)
  print 'Xmin,Xmax: ', xmin,xmax
  print xgrid 

  Q2grid = jamFF.get_Q2grid()
  Q2min = np.min(Q2grid)
  Q2max = np.max(Q2grid)
  print 'Q2min,Q2max: ', Q2min,Q2max
  print Q2grid

## Writes .info file

In [3]:
def writeinfo():

  H=[]
  H.append('SetDesc: '+desc)
  H.append('SetIndex: %i'%index)
  H.append('Authors: '+auth)
  H.append('Reference: '+ref)
  H.append('Format: lhagrid1')
  H.append('DataVersion: %i'%ver)

  H.append('Particle: %i'%particle)
  H.append('Flavors: '+str(flavors))
  H.append('OrderQCD: %i'%ordQCD)
  H.append('ForcePositive: 0')   # 0: do nothing 1: force to 0 if negative
  flavscheme = 'variable' # used for any variable flavor number scheme, massive or not
  H.append('FlavorScheme: '+flavscheme)
  H.append('NumFlavors: %i'%nfl)

  if errortype=='MC':
    nsets = nsamples+1   # number of posteriors + average
    errtype = 'replicas'  # LHAPDF-speak for MC method
  elif errortype=='HESSIAN': 
    nsets = 2*nsamples+2  # 2*eigenvalues + central fit
    errtype = 'hessian'
  H.append('NumMembers: %i'%nsets)
  H.append('ErrorType: '+errtype)
  H.append('ErrorConfLevel: %f'%cl)
  H.append('XMin: %e'%xmin)
  H.append('XMax: %e'%xmax)
  H.append('QMin: %e'%Qmin)
  H.append('QMax: %e'%Qmax)

  H.append('MZ: %f'%mZ)
  H.append('MUp: %f'%mu)
  H.append('MDown: %f'%md)
  H.append('MStrange: %f'%ms)
  H.append('MCharm: %f'%mc)
  H.append('MBottom: %f'%mb)
  H.append('MTop: %f'%mt)

  H.append('AlphaS_MZ: %f'%jamFF.get_alphaS(mZ**2))
  H.append('AlphaS_OrderQCD: %f'%ordQCD)
  H.append('AlphaS_Type: ipol')
  H.append('AlphaS_Qs: '+str(Rgrid.tolist()))
  alphas = []
  for Q in Rgrid:
    alphas.append(jamFF.get_alphaS(Q**2))
  H.append('AlphaS_Vals: '+str(alphas))
  H.append('AlphaS_Lambda4: %f'%jamFF.get_alphaS(mc**2))
  H.append('AlphaS_Lambda5: %f'%jamFF.get_alphaS(mb**2))

  for l in H: print l 

## Write a single .dat file

In [4]:
def writedat(member):
  
  # headers
  if member==0: 
    type='central'
  else:
    type='replica'
  H = []
  H.append('PdfType: '+type)
  H.append('Format: lhagrid1')
  H.append('---')  
  for l in H: print l
    
  # prints out x and Q grids  
  xvals = ["%10.4e" % x for x in ygrid]
  print ' '.join(xvals)
  Qvals = ["%10.4e" % Q for Q in Rgrid]
  print ' '.join(Qvals)
  flavs = ["%2i" % f for f in flavors]
  print ' '.join(flavs)
  
  # prints out the x*PDF(x) grid
  npart = len(partons)
    
  full_dist = []
  ix = 0
  iQ = 0
  il = 0
  for x in ygrid: 
    for Q in Rgrid:
      if member==0:  # Avreraged distribution
        FF = np.array([0 for n in range(npart)])
        for i in range(nsamples): 
          FF = FF + np.array([jamFF.get_XF(i,f,x,Q**2) for f in partons])
        FF = FF/nsamples
      else:   # member-th posterior 
        FF = [jamFF.get_XF(member-1,f,x,Q**2) for f in partons]
        #if Q < mb: FF[0] = FF[9] = 0
        #if Q < mc: FF[1] = FF[8] = 0
      if Q < mb and len(FF)>4: FF[4] = 0
      if Q < mc and len(FF)>3: FF[3] = 0
      #FF.append([x,Q])
      full_dist.append(FF)
  for l in full_dist: 
    out = ''
    for i in range(len(l)):
      out += '{:12.5e} '.format(l[i])
    print out

  print '---'  

 

## Writes all the .info and .dat files in one folder

In [5]:
def create_LHAPDF_grid(setname):
  
  print "Generating grids for "+setname
  
  start = os.getcwd()

  # Creates and navigates to the right folder
  if not os.path.exists('GRIDS/'+setname):
      os.makedirs('GRIDS/'+setname)
  os.chdir('GRIDS/'+setname)

  stdout = sys.stdout 
  
  # Writes the .info file
  f = open(setname+'.info','w')   
  sys.stdout = f    # <---------- very fragile solution, needs improvement
  writeinfo()
  f.close()
  sys.stdout=stdout

  for igrid in range(nsamples+1):
    gridname = '{}_{:04d}.dat'.format(setname,igrid)
    f = open(gridname,'w')
    sys.stdout = f   # <---------- very fragile solution, needs improvement
    writedat(igrid)
    f.close()
    sys.stdout=stdout
  
  # goes back to the initial folder
  os.chdir(start)



## log(1-z) grid generating functions

May not be needed, in the end, but I leave them here for now

In [6]:
def log_grid(x,a,d):    # logarithmic grid spacing function
  return np.log((1-x)**a+d)
  
def inv_log_grid(y,a,d):   # undoes the log grid 
  return 1-(np.exp(y)-d)**(1/a)

# GRID PRODUCTION

In [9]:
# Select JAM analysis
#jam_analysis = "../python/JAM16/FFpion"
jam_analysis = PATH+"JAM15/T3PPDF"
jamFF=JAMLIB(jam_analysis)


# Select LHAPDF set name for given JAM hadron
# and set the user-level appropriate LHAPDF flags 
# (see "SET LEVEL" in https://lhapdf.hepforge.org/config.html )
#
# NOTE:
#  
# setname = name of the set as it should appear in LHAPDF
# Particle = the hadron to which the PDFs belong, or the FF fragments into, see
#   http://pdg.lbl.gov/2015/reviews/rpp2015-rev-monte-carlo-numbering.pdf
#   For protons, neutrons, use format ±n nr nL nq1 nq2 nq3 nJ
#   E.g, proton = 2212 (u; u; d; spin 1/2 --> nJ=2*s+1)
#   For nuclei, use LZZZAAAI, see point 14 in PDG standard:
#   L nonzero is for hypernuclei, I represent isomer levels with
#   I = 0 the ground state, and Z and A should be self-explanatory.
#   Examples: 2212=proton   211=pi+  321=K+
# desc = Description of teh contents of the file - free form
# auth = Authors of the fit / set
# ver = version number: -1=beta; 1,2,3, ... official version, and updated/debugged ones if needed
# index = internal LHAPDF number scheme. Should uniquely identify the set, but important only if
#         set is to be included in teh official repository. Ask Andy Buckley <andy.buckley@cern.ch>
#         or the LHAPDF developers <lhapdf@projects.hepforge.org> for detail.
#         Format: index = nnxxx
#         nn (or nnn): assigned by Andy [LHAPDF]; for example 16=JAM fits - this to be discussed with Andy
#         xxx (or xx): up to us --> I would suggest 000-002 for JAM15; 003-004 for JAM16 FF; and so on as needed


# JAM15 - prelim grids

if jam_analysis==PATH+'JAM15/PPDF':
  setname = "JAM15_PPDF_Ceven"
  particle = 2212
  desc = "'JAM15 polarized PDFs of the proton. NLO fit (DIS only data). \
Flavors 901-905: q+ == q+qbar with q=d,u,s,c,b; Flavor 21: gluon. \
mem=0 => average of posteriors; mem=1-%d => PPDF posteriors.'"%jamFF.npos
  #Warning: only q^+ = q+qbar combination are physically meaningful; q-qbar=0 identically in this set.'"%jamFF.npos
  auth = "'[JAM collaboration] Sato N, Melnitchouk W, Kuhn S E, Ethier J J, Accardi A'"
  ref = "'Phys.Rev. D93 (2016) 114017 - www.jlab.org/jam - github.com/JeffersonLab/JAMLIB/tree/master/LHAPDF'"
  ver = -1   
  index = 16000
  partons = ['dp','up','sp','cp','bp','g']
  flavors = [901, 902, 903, 904, 905, 21]
  #           d+   u+   s+   c+   b+   g
if jam_analysis==PATH+'JAM15/T3PPDF':
  setname = "JAM15_T3PPDF_Ceven"
  particle = 2212
  desc = "'JAM15 polarized twist-3 PDFs of the proton. NLO fit (DIS only data). \
Flavors 901-902: q+ == q+qbar with q=d,u. \
mem=0 => average of posteriors; mem=1-%d => PPDF posteriors.'"%jamFF.npos
  auth = "'[JAM collaboration] Sato N, Melnitchouk W, Kuhn S E, Ethier J J, Accardi A'"
  ref = "'Phys.Rev. D93 (2016) 114017 - www.jlab.org/jam - github.com/JeffersonLab/JAMLIB/tree/master/LHAPDF'"
  ver = -1   
  index = 16001
  partons = ['dp','up']
  flavors = [901, 902]
  #          d+ u+

# JAM 16 - prelim grids
  
if jam_analysis==PATH+'JAM16/FFpion':
  setname = "JAM16_FF_pi_Ceven"
  particle = 211
  desc = "'JAM16 fragmentation functions of gluons and charge-conjugation-even quarks \
into pi+ mesons (identical to pi- by charge conjugation). NLO fit (DIS only data) \
Flavors 901-905: q+ == q+qbar with q=d,u,s,c,b; Flavor 21: gluon. \
mem=0 => average of posteriors; mem=1-%d => FF posteriors.'"%jamFF.npos
  auth = "'[JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A'"
  ref = "'arXiv:1609.00899 - www.jlab.org/jam - github.com/JeffersonLab/JAMLIB/tree/master/LHAPDF'"
  ver = -1   
  index = 16003
  partons = ['dp','up','sp','cp','bp','g']
  flavors = [901, 902, 903, 904, 905, 21]
  #          d+ u+ s+ c+ b+  g
elif jam_analysis==PATH+'JAM16/FFkaon':
  setname = "JAM16_FF_K_Ceven"
  particle = 321
  desc = "'JAM16 fragmentation functions of gluons and charge-conjugation-even quarks \
into K+ mesons (identical to K- by charge conjugation). NLO fit (DIS only data). \
Flavors 901-905: q+ == q+qbar with q=d,u,s,c,b; Flavor 21: gluon. \
mem=0 => average of posteriors; mem=1-%d => FF posteriors.'"%jamFF.npos
  auth = "'[JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A'"
  ref = "'arXiv:1609.00899 - www.jlab.org/jam - github.com/JeffersonLab/JAMLIB/tree/master/LHAPDF'"
  ver = -1 
  index = 16004
  partons = ['dp','up','sp','cp','bp','g']
  flavors = [901, 902, 903, 904, 905, 21]
  #          d+ u+ s+ c+ b+  g


########################################################## 
                                                           
     _   _    __  __ _     ___ ____                        
    | | / \  |  \/  | |   |_ _| __ )                       
 _  | |/ _ \ | |\/| | |    | ||  _ \                       
| |_| / ___ \| |  | | |___ | || |_) |                      
 \___/_/   \_\_|  |_|_____|___|____/                       
                                                           
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
                                                           
Authors:                                                   
Nobuo Sato         (Jefferson Lab)                         
Jake Ethier        (College of William and Mary)           
Wally Melnitchouk  (Jefferson Lab)                         
Alberto Accardi    (Hampton University and Jefferson Lab)  
########################################################## 
loading JAM15/T3PPDF tables [100%]


In [10]:
testjam(0.1,10.)

alphaS     =  0.24654958796
no. of posteriors =  200
xFF(ipos=0) =  0.0194832094343
Xmin,Xmax:  0.001 1.0
[ 0.001       0.01109091  0.02118182  0.03127273  0.04136364  0.05145455
  0.06154545  0.07163636  0.08172727  0.09181818  0.10190909  0.112
  0.12209091  0.13218182  0.14227273  0.15236364  0.16245455  0.17254545
  0.18263636  0.19272727  0.20281818  0.21290909  0.223       0.23309091
  0.24318182  0.25327273  0.26336364  0.27345455  0.28354545  0.29363636
  0.30372727  0.31381818  0.32390909  0.334       0.34409091  0.35418182
  0.36427273  0.37436364  0.38445455  0.39454545  0.40463636  0.41472727
  0.42481818  0.43490909  0.445       0.45509091  0.46518182  0.47527273
  0.48536364  0.49545455  0.50554545  0.51563636  0.52572727  0.53581818
  0.54590909  0.556       0.56609091  0.57618182  0.58627273  0.59636364
  0.60645455  0.61654545  0.62663636  0.63672727  0.64681818  0.65690909
  0.667       0.67709091  0.68718182  0.69727273  0.70736364  0.71745455
  0.72754545  0.7376363

In [11]:
# COLLECTS INFO ON JAM FIT

# QCD setup
ordQCD =1  # 1: NLO
nfl = 5

# This is for JAM FF or PDF, for others, need to change
errortype = 'MC'      # MC = Monte Carlo method --> nsamples = no. of posteriors
nsamples = jamFF.npos # HESSIAN = Hessian --> nsamples = no. free parameters
cl = 68.3             # confidence level

# Collects info from JAM fits
xgrid = jamFF.get_Xgrid()
xmin = np.min(xgrid)
xmax = np.max(xgrid)
Qgrid = jamFF.get_Q2grid()**0.5
Qmin = np.min(Qgrid)
Qmax = np.max(Qgrid)
mu = 0.
md = 0.
ms = 0.
mc = jamFF.mc2**0.5
mb = jamFF.mb2**0.5
mt = jamFF.mt2**0.5
mZ = jamFF.mZ2**0.5

# Creates the momentum fraction grid (ygrid), and Q-scale grid (Rgrid)
# that can be different from JAM grids, in principle

#a=0.7
  #d=100.
  #min,ymax = log_grid(xgrid[0],a,d),log_grid(xgrid[-1],a,d)
  #y=np.linspace(ymin,ymax,len(xgrid))
  #ygrid = inv_log_grid(y,a,d)
ygrid=xgrid

  #Rmin,Rmax=np.log(Qgrid[0]),np.log(Qgrid[-1])
  #nQ2=100
  #R = np.linspace(Rmin,Rmax,nQ2)
  #Rgrid = np.exp(R)
Rgrid = Qgrid

In [12]:
writeinfo()

SetDesc: 'JAM15 polarized twist-3 PDFs of the proton. NLO fit (DIS only data). Flavors 901-902: q+ == q+qbar with q=d,u. mem=0 => average of posteriors; mem=1-200 => PPDF posteriors.'
SetIndex: 16001
Authors: '[JAM collaboration] Sato N, Melnitchouk W, Kuhn S E, Ethier J J, Accardi A'
Reference: 'Phys.Rev. D93 (2016) 114017 - www.jlab.org/jam - github.com/JeffersonLab/JAMLIB/tree/master/LHAPDF'
Format: lhagrid1
DataVersion: -1
Particle: 2212
Flavors: [901, 902]
OrderQCD: 1
ForcePositive: 0
FlavorScheme: variable
NumFlavors: 5
NumMembers: 201
ErrorType: replicas
ErrorConfLevel: 68.300000
XMin: 1.000000e-03
XMax: 1.000000e+00
QMin: 1.000000e+00
QMax: 1.000000e+03
MZ: 91.187000
MUp: 0.000000
MDown: 0.000000
MStrange: 0.000000
MCharm: 1.430000
MBottom: 4.300000
MTop: 172.900000
AlphaS_MZ: 0.118000
AlphaS_OrderQCD: 1.000000
AlphaS_Type: ipol
AlphaS_Qs: [1.0, 1.1242100350620865, 1.263848202934298, 1.4208308325339223, 1.43, 1.5973122800602544, 1.79571449437164, 2.0187602546790386, 2.269510536

In [13]:
writedat(1)

PdfType: replica
Format: lhagrid1
---
1.0000e-03 1.1091e-02 2.1182e-02 3.1273e-02 4.1364e-02 5.1455e-02 6.1545e-02 7.1636e-02 8.1727e-02 9.1818e-02 1.0191e-01 1.1200e-01 1.2209e-01 1.3218e-01 1.4227e-01 1.5236e-01 1.6245e-01 1.7255e-01 1.8264e-01 1.9273e-01 2.0282e-01 2.1291e-01 2.2300e-01 2.3309e-01 2.4318e-01 2.5327e-01 2.6336e-01 2.7345e-01 2.8355e-01 2.9364e-01 3.0373e-01 3.1382e-01 3.2391e-01 3.3400e-01 3.4409e-01 3.5418e-01 3.6427e-01 3.7436e-01 3.8445e-01 3.9455e-01 4.0464e-01 4.1473e-01 4.2482e-01 4.3491e-01 4.4500e-01 4.5509e-01 4.6518e-01 4.7527e-01 4.8536e-01 4.9545e-01 5.0555e-01 5.1564e-01 5.2573e-01 5.3582e-01 5.4591e-01 5.5600e-01 5.6609e-01 5.7618e-01 5.8627e-01 5.9636e-01 6.0645e-01 6.1655e-01 6.2664e-01 6.3673e-01 6.4682e-01 6.5691e-01 6.6700e-01 6.7709e-01 6.8718e-01 6.9727e-01 7.0736e-01 7.1745e-01 7.2755e-01 7.3764e-01 7.4773e-01 7.5782e-01 7.6791e-01 7.7800e-01 7.8809e-01 7.9818e-01 8.0827e-01 8.1836e-01 8.2845e-01 8.3855e-01 8.4864e-01 8.5873e-01 8.6882e-01 8.789

In [14]:
# CREATE THE GRIDS

create_LHAPDF_grid(setname)

Generating grids for JAM15_T3PPDF_Ceven
